In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import gensim.models.keyedvectors as word2vec
import gc

/home/andy/anaconda3/envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [29]:
path = './data/'
EMBEDDING_FILE=f'{path}glove.6B.50d.txt'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [34]:
# try 50, 100, 200 and 300 
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
# try 50, 100, 200 and 300 
maxlen = 200 # max number of words in a comment to use

In [4]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [5]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [31]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [32]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940498, 0.6441043)

In [35]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

Load embedding matrix from [kernal](https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge/notebook)

In [10]:
def loadEmbeddingMatrix(typeToLoad):
        #load different embedding file from Kaggle depending on which embedding 
        #matrix we are going to experiment with
        if(typeToLoad=="glove"):
            EMBEDDING_FILE=f'{path}glove.twitter.27B.25d.txt'
            embed_size = 25
        elif(typeToLoad=="word2vec"):
            word2vecDict = word2vec.KeyedVectors.load_word2vec_format(f'{path}GoogleNews-vectors-negative300.bin', binary=True)
            embed_size = 300
        elif(typeToLoad=="fasttext"):
            EMBEDDING_FILE=f'{path}wiki.simple.vec'
            embed_size = 300

        if(typeToLoad=="glove" or typeToLoad=="fasttext" ):
            embeddings_index = dict()
            #Transfer the embedding weights into a dictionary by iterating through every line of the file.
            f = open(EMBEDDING_FILE)
            for line in f:
                #split up line into an indexed array
                values = line.split()
                #first index is word
                word = values[0]
                #store the rest of the values in the array as a new array
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs #50 dimensions
            f.close()
            print('Loaded %s word vectors.' % len(embeddings_index))
        else:
            embeddings_index = dict()
            for word in word2vecDict.wv.vocab:
                embeddings_index[word] = word2vecDict.word_vec(word)
            print('Loaded %s word vectors.' % len(embeddings_index))
            
        gc.collect()
        #We get the mean and standard deviation of the embedding weights so that we could maintain the 
        #same statistics for the rest of our own random generated weights. 
        all_embs = np.stack(list(embeddings_index.values()))
        emb_mean,emb_std = all_embs.mean(), all_embs.std()
        
        nb_words = len(tokenizer.word_index)
        #We are going to set the embedding size to the pretrained dimension as we are replicating it.
        #the size will be Number of Words in Vocab X Embedding Size
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
        gc.collect()

        #With the newly created embedding matrix, we'll fill it up with the words that we have in both 
        #our own dictionary and loaded pretrained embedding. 
        embeddedCount = 0
        for word, i in tokenizer.word_index.items():
            i-=1
            #then we see if this word is in glove's dictionary, if yes, get the corresponding weights
            embedding_vector = embeddings_index.get(word)
            #and store inside the embedding matrix that we will train later on.
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
                embeddedCount+=1
        print('total embedded:',embeddedCount,'common words')
        
        del(embeddings_index)
        gc.collect()
        
        #finally, return the embedding matrix
        return embedding_matrix

Try word2vec for multi then run all for multi (4x6) and average

In [11]:
embedding_matrix = loadEmbeddingMatrix('word2vec')

Loaded 3000000 word vectors.
total embedded: 66078 common words


For embedings trained from train data see this [kernal](https://www.kaggle.com/sbongo/for-beginners-tackling-toxic-using-keras/notebook)

In [37]:
resu = np.zeros(shape=(X_te.shape[0],6))

for i in range(6):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
#     x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)
    x = Bidirectional(GRU(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    print(f'Fitting {list_classes[i]}')
    model.fit(X_t, y[:,i], batch_size=32, epochs=2)
    
    print(f'Predicting {list_classes[i]}')
    resu[:,i] = model.predict([X_te], batch_size=1024, verbose=1)[:,0]
    print('\n')

    

Fitting toxic
Epoch 1/2
159571/159571 [==============================] - 447s 3ms/step - loss: 0.1168 - acc: 0.9571
Epoch 2/2
159571/159571 [==============================] - 444s 3ms/step - loss: 0.0873 - acc: 0.9667
Predicting toxic
153164/153164 [==============================] - 53s 347us/step


Fitting severe_toxic
Epoch 1/2
159571/159571 [==============================] - 448s 3ms/step - loss: 0.0270 - acc: 0.9903
Epoch 2/2
159571/159571 [==============================] - 446s 3ms/step - loss: 0.0213 - acc: 0.9907
Predicting severe_toxic
153164/153164 [==============================] - 53s 347us/step


Fitting obscene
Epoch 1/2
159571/159571 [==============================] - 452s 3ms/step - loss: 0.0643 - acc: 0.9773
Epoch 2/2
159571/159571 [==============================] - 452s 3ms/step - loss: 0.0461 - acc: 0.9826
Predicting obscene
153164/153164 [==============================] - 53s 348us/step


Fitting threat
Epoch 1/2
159571/159571 [==============================] - 454s 

In [13]:
preds = model.predict([X_te], batch_size=1024, verbose=1)

preds.shape

153164/153164 [==============================] - 162s 1ms/step


(153164, 1)

In [28]:
resu[:,0] = preds[:,0]

preds[:,0].shape

(153164,)

In [38]:
sample_submission = pd.read_csv(f'{path}sample_submission.csv')
sample_submission.shape
sample_submission[list_classes] = resu
sample_submission.to_csv('multi_submission.csv', index=False)

Use GRU for Stox, threat and IDH 

Use LSTM for tox, obs and ins

tox is the lowest atm so find optimal params for lstm

### Todo

[x] Run the full notebook 

[ ] try different combos of max features and embed size with a subset of data to find a good combo 

[ ] find the best combos for toxic, obscene and insult

[ ] try training a model for all using the multimodel preds in training data